In [57]:
import pandas as pd
import cv2 as cv

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import PoseLandmarkerResult

In [58]:
# Absolute Path to the Model
modelPath = '/home/matt/Documents/projects/swingAnalysis/swing-analysis-prototyping/models/pose_landmarker_full.task'

# Set up MediaPipe base options
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with video mode on
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=modelPath),
    running_mode=VisionRunningMode.IMAGE
)

In [59]:
# Read in the swing segmentation data
df = pd.read_pickle('../data/golfDB.pkl')
df = df[df['view'] != 'other']
df = df[df['slow'] == 0]
df

,id,youtube_id,player,sex,club,view,slow,events,bbox,split
0,0,f1BWA5F87Jc,SANDRA GAL,f,driver,down-the-line,0,"[408, 455, 473, 476, 490, 495, 498, 501, 514, ...","[0.09765625000000001, 0.006944444444444444, 0....",3
2,2,tA1iotgtMyc,CHRIS DIMARCO,m,driver,down-the-line,0,"[521, 659, 678, 683, 692, 696, 698, 701, 715, ...","[0.165625, 0.0006944444444444445, 0.48359375, ...",3
4,4,wDCKLePrwHA,BROOKE HENDERSON,f,driver,down-the-line,0,"[157, 170, 183, 188, 197, 201, 205, 207, 220, ...","[0.11015625, 0.0006944444444444445, 0.4984375,...",3
6,6,iPuVhnI8pJU,NICK WATNEY,m,driver,down-the-line,0,"[246, 298, 310, 314, 324, 329, 332, 334, 351, ...","[0.1453125, 0.001388888888888889, 0.46796875, ...",2
8,8,-M5SITXMA2Y,CRISTIE KERR,f,driver,face-on,0,"[288, 317, 333, 335, 347, 352, 355, 357, 371, ...","[0.0007812500000000111, 0.0006944444444444445,...",4
...,...,...,...,...,...,...,...,...,...,...
1388,1388,GXn3A0IuWsE,ROGER CLEMENS,m,driver,face-on,0,"[211, 255, 265, 270, 276, 280, 284, 286, 297, ...","[0.12109375, 0.09583333333333334, 0.65703125, ...",1
1392,1392,66k8pcYkY3M,GARY WOODLAND,m,driver,down-the-line,0,"[61, 251, 267, 273, 287, 290, 293, 295, 308, 340]","[0.17734375, 0.0006944444444444445, 0.5140625,...",1
1394,1394,bXqJtcWylfQ,ARIYA JUTANUGARN,f,iron,down-the-line,0,"[301, 539, 555, 560, 568, 572, 575, 578, 595, ...","[0.171875, 0.0006944444444444445, 0.4140625, 0...",1
1396,1396,TtxGGIzrqWI,KENNY PERRY,m,driver,face-on,0,"[121, 394, 407, 414, 427, 431, 435, 437, 452, ...","[0.16796875, 0.0006944444444444445, 0.7171875,...",3


In [60]:
# Method for determining if a player is lefty or righty
def left_or_right(pose_landmarks, view):
    # Extract the shoulder data from the landmarks
    leftShoulder = pose_landmarks[11]
    leftAnkle = pose_landmarks[27]
    rightShoulder = pose_landmarks[12]
    rightAnkle = pose_landmarks[28]

    # Determine which view is being used to know what comparison to make
    if (view == 'down-the-line'):
        # Check the depth of the points using z point
        # The front shoulder will have a greater value
        if (leftShoulder.z > rightShoulder.z and leftAnkle.z > rightAnkle.z):
            return 'right'
        elif (leftShoulder.z < rightShoulder.z and leftAnkle.z < rightAnkle.z):
            return 'left'
    elif (view == 'face-on'):
        # Check the depth of the points using x point
        # The front shoulder will have a greater value
        if (leftShoulder.x > rightShoulder.x and leftAnkle.x > rightAnkle.x):
            return 'right'
        elif (leftShoulder.x < rightShoulder.x and leftAnkle.x < rightAnkle.x):
            return 'left'
    return 'ERROR'

In [61]:
handedness = []

# Run the Landmarker
with PoseLandmarker.create_from_options(options) as landmarker:
    # Iterate through each player in the dataframe
    for rowIdx, row in enumerate(df.iterrows()):
        print(f'Row {rowIdx+1}/{len(df)}')
        # Set up OpenCV Video Capture
        cap = cv.VideoCapture(f'../data/videos_160/{row[1]["id"]}.mp4')
        events = row[1]['events']
        events = (events - events[0])[1:-1]

        swingStart = events[0]
        
        # Jump to the frames where first event is
        cap.set(cv.CAP_PROP_POS_FRAMES, swingStart)
        success, frame = cap.read()
        if not success:
            break
        
        # Convert the image to an mp image
        mpImage = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Get the pose for the image
        pose_landmarker_result = landmarker.detect(mpImage)
        
        # Get the handedness and append the result to the list
        if pose_landmarker_result.pose_landmarks:
            handedness.append(
                left_or_right(
                    pose_landmarker_result.pose_landmarks[0],
                    row[1]['view']
                )
            )
        else:
            print('Error Processing')
            print(row)
            handedness.append('ERROR')

    cap.release()
    cv.destroyAllWindows()

I0000 00:00:1750868758.330153   74429 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750868758.334118  119973 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.3-1ubuntu2), renderer: zink Vulkan 1.4(NVIDIA GeForce RTX 4070 (NVIDIA_PROPRIETARY))
W0000 00:00:1750868758.383912  119976 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750868758.418709  119986 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Row 1/556
Row 2/556
Row 3/556
Row 4/556
Row 5/556
Row 6/556
Row 7/556
Row 8/556
Row 9/556
Row 10/556
Row 11/556
Row 12/556
Row 13/556
Row 14/556
Row 15/556
Row 16/556
Row 17/556
Row 18/556
Row 19/556
Row 20/556
Row 21/556
Row 22/556
Row 23/556
Row 24/556
Row 25/556
Row 26/556
Row 27/556
Row 28/556
Row 29/556
Row 30/556
Row 31/556
Row 32/556
Row 33/556
Row 34/556
Row 35/556
Row 36/556
Row 37/556
Row 38/556
Row 39/556
Row 40/556
Row 41/556
Row 42/556
Row 43/556
Row 44/556
Row 45/556
Row 46/556
Row 47/556
Row 48/556
Row 49/556
Row 50/556
Row 51/556
Row 52/556
Row 53/556
Error Processing
(124, id                                                          124
youtube_id                                          iuWywRQ2HCs
player                                                JOHN DALY
sex                                                           m
club                                                    fairway
view                                              down-the-line
slow               

In [62]:
# Assign handedness data to dataframe and filter out errors
df['handedness'] = handedness
df = df[df['handedness'] != 'ERROR']
df['handedness'].value_counts()

handedness
right    524
left      14
Name: count, dtype: int64

In [63]:
# Save the df to a file so that the handedness can be checked
df.to_pickle('../data/GolfDB_Filtered.pkl')